# GPX Viewer

This app lets you to display a track from a GPX file recorded with a GPS device.

In [1]:
import sys, os
from ipywidgets import Button, HTML, HBox, FileUpload, Label, Output, Layout, Image

sys.path.append(os.getcwd() + '/src')
import sailing_analytics
import trail_mapping

In [2]:
tools = ["voila", "ipyleaflet", "ipywidgets", "bqplot"]
logos = []
for tool in tools:
    with open(f'./img/{tool}.png', 'rb') as f:
        image = f.read()
    img = Image(value=image, format='png',layout=Layout(padding='10px'))
    logos.append(img)
HBox([Label(value='Powered by:')] + logos, layout=Layout(flex_flow='row', align_items='center'))

In [7]:
"""
create the output widget to place the results
"""
global out 
out = Output()

out_bottom = Output()

In [8]:
def show_uploader():
    uploader = FileUpload(accept='.gpx', multiple=False)

    def handle_upload(change):
        # keep only the last file
        # TODO: check if this should be fixed in FileUpload widget
        # when multiple=False
        *_, (_, f) = change['new'].items()
        gpx_content = f['content'].decode('utf-8')
        out.clear_output()
        with StringIO(gpx_content) as gpx_file:
            session = sailing_analytics.SailingSession(gpx_file)
            with out_bottom:
                show_map_polar_buttons(session, file=gpx_content)

    uploader.observe(handle_upload, names='value')
    display(uploader)

In [9]:
def show_examples():
    example_folder = "./examples"
    examples = [f for f in os.listdir(example_folder) if f.endswith('.gpx')]
    
    def create_example(name):
        filename = os.path.join(example_folder, name)
        
        @out.capture()
        def on_example_clicked(change):
            out.clear_output()
            with out_bottom:
                global session
                session = sailing_analytics.SailingSession(filename)
                show_map_polar_buttons(session, filename)
                trail_mapping.plot_gpx(filename)
    
        button = Button(description=os.path.splitext(name)[0])
        button.on_click(on_example_clicked)
        return button
    
    buttons = [create_example(example) for example in examples]
    line = HBox(buttons, layout=Layout(flex_flow='row', align_items='center'))
    display(line)

In [10]:
def show_map_polar_buttons(session, filename=""):
    def map_button():
        @out.capture()
        def map_button_clicked(change):
            out.clear_output()
            with out:
                session.map()

        button = Button(description="Show Map")
        button.on_click(map_button_clicked)
        return button

    def polar_button():
        @out.capture()
        def polar_button_clicked(change):
            out.clear_output()
            with out:
                session.polar()

        button = Button(description="Show Polar")
        button.on_click(polar_button_clicked)
        return button
    
    def trail_button():
        @out.capture()
        def trail_button_clicked(change):
            out.clear_output()
            with out:
                trail_mapping.plot_gpx(filename)

        button = Button(description="Show Trail")
        button.on_click(trail_button_clicked)
        return button

    buttons = [map_button(), polar_button(), trail_button()]
    line = HBox(buttons, layout=Layout(flex_flow='row', align_items='center'))
    display(line)


In [11]:
out

Output()

In [13]:
show_examples()

If you don't have a GPX file, try with one of the following examples:

In [14]:
show_uploader()

FileUpload(value={}, accept='.gpx', description='Upload')

In [15]:
out_bottom

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': "HBox(children=(Button(description='Sho…

In [19]:
session.df

,lon,lat,alt,time,distance_step,distance_cumulative,time_diff,time_elapsed_sec,bearing,speed_ms,speed_kts,speed_kts_capped,vmg_ms,vmg_kts,upwind,tack_raw,is_moving,tack,segment
0,-79.337713,43.637683,69.354797,2022-09-06 18:38:08-04:00,0.000000,0.000000,1.0,0.0,0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000,0.0,1,1.0,1.0,1.0
1,-79.337727,43.637763,75.661343,2022-09-06 18:38:09-04:00,8.976909,8.976909,1.0,1.0,352.666658,8.976909,17.449675,15.000000,-1.815287,-3.528628,-1.0,1,1.0,1.0,1.0
2,-79.337704,43.637795,75.713284,2022-09-06 18:38:10-04:00,3.984548,12.961457,1.0,2.0,27.853232,3.984548,7.745324,7.745324,-2.907144,-5.651022,-1.0,1,1.0,1.0,1.0
3,-79.337675,43.637761,75.604336,2022-09-06 18:38:11-04:00,4.388424,17.349881,1.0,3.0,148.176776,4.388424,8.530393,8.530393,-0.973983,-1.893267,-1.0,-1,1.0,1.0,1.0
4,-79.337701,43.637731,76.013088,2022-09-06 18:38:13-04:00,3.972911,21.322792,2.0,5.0,212.107829,1.986455,3.861351,3.861351,1.546116,3.005402,1.0,-1,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1063,-79.337052,43.641233,79.056973,2022-09-06 19:33:15-04:00,2.613834,2951.278402,2.0,3307.0,352.890396,1.306917,2.540438,2.540438,-0.269277,-0.523432,-1.0,1,1.0,1.0,15.0
1064,-79.337045,43.641261,78.463711,2022-09-06 19:33:17-04:00,3.183882,2954.462283,2.0,3309.0,11.478473,1.591941,3.094478,3.094478,-0.807456,-1.569565,-1.0,1,1.0,1.0,15.0
1065,-79.337027,43.641276,78.532009,2022-09-06 19:33:18-04:00,2.204099,2956.666383,1.0,3310.0,40.944872,2.204099,4.284416,4.284416,-1.907745,-3.708351,-1.0,1,1.0,1.0,15.0
1066,-79.337023,43.641297,77.820400,2022-09-06 19:33:20-04:00,2.356362,2959.022745,2.0,3312.0,7.229883,1.178181,2.290195,2.290195,-0.520725,-1.012206,-1.0,1,1.0,1.0,15.0


In [35]:
(session.get_gpx_points_list(session.gpx)[0])[0:3]

[GPXTrackPoint(43.637682991142974, -79.33771289892432, elevation=69.35479736328125, time=datetime.datetime(2022, 9, 6, 22, 38, 8, tzinfo=SimpleTZ("Z"))),
 GPXTrackPoint(43.637763122137294, -79.33772714815971, elevation=75.66134262084961, time=datetime.datetime(2022, 9, 6, 22, 38, 9, tzinfo=SimpleTZ("Z"))),
 GPXTrackPoint(43.63779480573128, -79.33770401410696, elevation=75.71328353881836, time=datetime.datetime(2022, 9, 6, 22, 38, 10, tzinfo=SimpleTZ("Z")))]

In [36]:
(session.get_gpx_points_list(session.gpx)[1])[0:3]

[[-79.33771289892432,
  43.637682991142974,
  69.35479736328125,
  datetime.datetime(2022, 9, 6, 18, 38, 8, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>)],
 [-79.33772714815971,
  43.637763122137294,
  75.66134262084961,
  datetime.datetime(2022, 9, 6, 18, 38, 9, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>)],
 [-79.33770401410696,
  43.63779480573128,
  75.71328353881836,
  datetime.datetime(2022, 9, 6, 18, 38, 10, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>)]]